# Court Data Extractor Demo

This notebook demonstrates how to use different modules from the Court Data Extractor package.

## Setup

First, let's import all necessary modules and create instances of our extractors.

In [6]:
import os
import sys
import pandas as pd

# Add the project root directory to Python path
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
sys.path.append(project_root)

from src.articles import ArticlesExtractor
from src.gender import GenderExtractor
from src.districts import MunicipalityExtractor
from src.punishments import PunishmentExtractor

# Initialize extractors
articles_extractor = ArticlesExtractor(remove_duplicates=True)
gender_extractor = GenderExtractor(russian_names_db=False)
municipality_extractor = MunicipalityExtractor()
punishment_extractor = PunishmentExtractor(api_key="your api key")

## 1. Articles Extractor

The Articles Extractor helps parse legal articles, their parts, and subparts from court decision texts.

In [3]:
# Example 1: Process single string
test_string = "Губаев Борис Магомедович - ст.159 ч.2 УК РФ"
result = articles_extractor.process_string(test_string)
print("Single string processing result:")
print(result)

# Example 2: Process DataFrame
data = {
    'text_column': [
        "Губаев Борис Магомедович - ст.159 ч.2 УК РФ",
        "ст. 20.1 КоАП; ст. 19.3 ч.1 КоАП",
        "ст. 105 ч.1 п.а УК; ст. 111 ч.2 УК"
    ]
}
df = pd.DataFrame(data)

results = articles_extractor.process_dataframe(df, 'text_column', parallel=True, n_workers=2)
print("\nDataFrame processing results:")
for i, result in enumerate(results):
    print(f"\nRow {i+1}:")
    print(result)

Single string processing result:
[{'person': 'Person 1', 'articles': [{'article': '159', 'part': '2', 'subpart': None}], 'code_type': 'CRIMINAL'}]

DataFrame processing results:

Row 1:
[{'person': 'Person 1', 'articles': [{'article': '159', 'part': '2', 'subpart': None}], 'code_type': 'CRIMINAL'}]

Row 2:
[{'person': 'Person 1', 'articles': [{'article': '20.1', 'part': None, 'subpart': None}, {'article': '19.3', 'part': '1', 'subpart': None}], 'code_type': 'ADMIN'}]

Row 3:
[{'person': 'Person 1', 'articles': [{'article': '105', 'part': '1', 'subpart': ['а']}, {'article': '111', 'part': '2', 'subpart': None}], 'code_type': 'CRIMINAL'}]


## 2. Gender Extractor

The Gender Extractor helps determine the gender of defendants from their full names.

In [4]:
# Example: Extract gender from text
text = "Волостных Владислав Витальевич - ст.291 ч.3; ст.222 ч.1; ст.290 ч.5 п.в; ст.290 ч.5 п.в; ст.290 ч.5 п.в УК РФ"
result = gender_extractor.extract_genders(text)
print("Gender extraction result:")
print(result)

# Note: Results show (name, gender) where gender can be:
# M - male
# F - female
# U - undefined
# C - contradiction

Gender extraction result:
[('Волостных Владислав Витальевич', 'M')]


## 3. Municipality Extractor

The Municipality Extractor helps determine the region and municipality for a given court code.

In [5]:
# Example 1: Get municipality for single court code
court_code = "61RS0006"
region, municipality, oktmo = municipality_extractor.get_municipality(court_code)
print(f"Court code: {court_code}")
print(f"Region: {region}")
print(f"Municipality: {municipality}")
print(f"OKTMO: {oktmo}")

# Example 2: Process DataFrame with court codes
data = {
    'court_code': ['61RS0006', '61RS0007', '61RS0008']
}
df = pd.DataFrame(data)
df = municipality_extractor.process_dataframe(df, 'court_code')
print("\nDataFrame processing result:")
print(df)

Court code: 61RS0006
Region: Ростовская
Municipality: Городской округ Город Ростов-на-Дону
OKTMO: 60701000

DataFrame processing result:
  court_code      region                          municipality     oktmo
0   61RS0006  Ростовская  Городской округ Город Ростов-на-Дону  60701000
1   61RS0007  Ростовская  Городской округ Город Ростов-на-Дону  60701000
2   61RS0008  Ростовская  Городской округ Город Ростов-на-Дону  60701000


## 4. Punishment Extractor

The Punishment Extractor helps extract structured information about punishments from court decision texts.

In [7]:
input_string = """Шестакова Александра Владимировича признать виновным в совершении преступлений, предусмотренных п. «з» ч.2 ст.111, п. 
«а» ч.3 ст.158 Уголовного кодекса Российской Федерации и назначить ему наказание: - по п. «з» ч.2 ст.111УК РФ – в виде лишения свободы на 
срок три года; - по п. «а» ч.3 ст.158 УК РФ – в виде лишения свободы на срок два года. На основании ч.3 ст. 69 Уголовного кодекса Российской 
Федерации по совокупности преступлений путем частичного сложения наказаний окончательно назначить Шестакову Александру Владимировичу наказание 
в виде лишения свободы на срок четыре года. В соответствии со ст. 70 Уголовного кодекса Российской Федерации, по совокупности приговоров, 
к наказанию, назначенному по настоящему приговору, частично присоединить неотбытую часть наказания по приговору от 18.05.2021 Уватского районного 
суда Тюменской области и окончательно назначить Шестакову Александру Владимировичу наказание в виде лишения свободы на срок 4 (четыре) года 6 (шесть)
месяцев, с отбыванием в исправительной колонии общего режима. Меру пресечения в виде подписки о невыезде и надлежащем поведении Шестакову 
Александру Владимировичу изменить на меру пресечения в виде заключения под стражу, взять его под стражу немедленно в зале суда. 
Срок наказания Шестакову Александру Владимировичу исчислять со дня вступления настоящего приговора в законную силу. В соответствии с п. "б" ч. 
3.1 ст. 72 Уголовного кодекса Российской Федерации время содержания под стражей Шестакову Александру Владимировичу с 18 мая 2023 года до дня 
вступления настоящего приговора в законную силу зачесть в срок лишения свободы из расчета один день содержания под стражей за полтора дня отбывания 
наказания в исправительной колонии общего режима."""

initial_string, res = punishment_extractor.find_punishemtns(input_string)

print("Punishment extraction results:")
print(res)

Punishment extraction results:
{'Шестаков Александр Владимирович': {'1': {'punishment': 'лишение свободы на определенный срок', 'type': 'колония общего режима', 'severity': {'years': 4, 'months': 6, 'rubles': 0, 'days': 0, 'hours': 0}}}}
